In [1]:
import pandas as pd

# Dataset importaion

In [5]:
hotel_train_data= pd.read_csv('/Users/Datasets/HotelReviews/train.csv')

# EDA

In [6]:
hotel_train_data.shape

(38932, 5)

In [7]:
hotel_train_data.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [8]:
hotel_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38932 entries, 0 to 38931
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   User_ID       38932 non-null  object
 1   Description   38932 non-null  object
 2   Browser_Used  38932 non-null  object
 3   Device_Used   38932 non-null  object
 4   Is_Response   38932 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB


In [9]:
hotel_train_data.isna().sum()

User_ID         0
Description     0
Browser_Used    0
Device_Used     0
Is_Response     0
dtype: int64

In [12]:
hotel_train_data = hotel_train_data[['Description','Is_Response']]

In [11]:
hotel_train_data

,Description,Is_Response
0,The room was kind of clean but had a VERY stro...,not happy
1,I stayed at the Crown Plaza April -- - April -...,not happy
2,I booked this hotel through Hotwire at the low...,not happy
3,Stayed here with husband and sons on the way t...,happy
4,My girlfriends and I stayed here to celebrate ...,not happy
...,...,...
38927,We arrived late at night and walked in to a ch...,happy
38928,The only positive impression is location and p...,not happy
38929,Traveling with friends for shopping and a show...,not happy
38930,The experience was just ok. We paid extra for ...,not happy


# Data Cleaning

In [13]:
import re
import string

In [24]:
# creat a function for cleaning
def data_clean(text):
    text =text.lower() # convert to lower
    text = re.sub('\[,*?\]','',text) # remove square bracket
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text)# remove punctuation
    text =re.sub('\W*\d\w*','',text) # reove digit
    text = re.sub('\n','',text)
    text =re.sub('[''""...]','',text)
    return text

In [25]:
clean_data = lambda x: data_clean(x)

In [26]:
hotel_train_data['cleaned'] = pd.DataFrame(hotel_train_data.Description.apply(clean_data))
hotel_train_data.head()

,Description,Is_Response,cleaned
0,The room was kind of clean but had a VERY stro...,not happy,the room was kind of clean but had a very stro...
1,I stayed at the Crown Plaza April -- - April -...,not happy,i stayed at the crown plaza april april th...
2,I booked this hotel through Hotwire at the low...,not happy,i booked this hotel through hotwire at the low...
3,Stayed here with husband and sons on the way t...,happy,stayed here with husband and sons on the way t...
4,My girlfriends and I stayed here to celebrate ...,not happy,my girlfriends and i stayed here to celebrate ...


# Model Training

In [27]:
from sklearn.model_selection import train_test_split
target = hotel_train_data.Is_Response
predictor= hotel_train_data.cleaned
X_train,X_test, Y_train, Y_test = train_test_split(predictor,target,test_size=0.1,random_state=225)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = TfidfVectorizer()
model = LogisticRegression(solver='lbfgs')

In [43]:
from sklearn.pipeline import Pipeline
model_pipeline = Pipeline([('vectorize',vectorizer),('model',model)])
model_pipeline.fit(X_train,Y_train)


Pipeline(steps=[('vectorize', TfidfVectorizer()),
                ('model', LogisticRegression())])

In [51]:
from sklearn. metrics import confusion_matrix,accuracy_score


predict_Y = model_pipeline.predict(X_test)

print(confusion_matrix(predict_Y,Y_test))
print(accuracy_score(predict_Y, Y_test)*100)
#import seaborn as sns
#sns.heatmap(confusion_matrix(predict_Y,Y_test), annot=True)

[[2417  305]
 [ 154 1018]]
88.21263482280432


# Test model

In [59]:
test = model_pipeline.predict(['I am not satisfied with the hotel'])
print(test)

['not happy']
